# Quick agent notebook interactive version

In [1]:
import sys
sys.path.append("../quick_agent/agent/")

In [2]:
from agent import root_agent # get root_agent built from the python script
from dotenv import load_dotenv
load_dotenv("../quick_agent/agent/.env") # get environment variables needed

# Set up to run interactively
import asyncio
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

session_service = InMemorySessionService()
session = await session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)

runner = Runner(app_name=APP_NAME, agent = root_agent, session_service=session_service)

async def call_agent_async(query:str, runner, user_id, session_id):
    """Send a query to the agent and print final response"""
    
    print(f"\n>>>User request: {query}.")
    content = types.Content(role='user', parts= [types.Part(text=query)])
    final_response = 'Agent did not produce a final response.'
    
    async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
        if event.is_final_response():
            if event.content and event.content.parts:
                final_response = event.content.parts[0].text
            elif event.actions and event.actions.escalate:
                final_response = f"Agent escalate {event.error_message or 'No specific error message.'}"
            break
    print(f"<<< Agent response: {final_response}")

USER_ID = 'user_001'
SESSION_ID = 'session_001'
APP_NAME = 'quick_agent'

True

In [10]:
# Run interactively
await call_agent_async('hello', runner, USER_ID, SESSION_ID)
await call_agent_async('what is the weather in new york?', runner, USER_ID, SESSION_ID)
await call_agent_async('what is the weather in DC?', runner, USER_ID, SESSION_ID)
await call_agent_async('what is the time in new york?', runner, USER_ID, SESSION_ID)


>>>User request: hello.
<<< Agent response: Hello! How can I help you with the weather today?


>>>User request: what is the weather in new york?.


<<< Agent response: The temperature in New York is 85 Fahrenheit.


>>>User request: what is the weather in DC?.


<<< Agent response: I am sorry, the weather information is not available for DC. Would you like me to check another city?


>>>User request: what is the time in new york?.
<<< Agent response: I am sorry, I can only provide weather information.

